In [1]:
from model4c import HRNetRefine
from relay_net import ReLayNet
from channelconfig import get_cfg_defaults
import json
import re
import torch
import inspect

In [2]:
class ModelParser():
   def __init__(self,model) -> None:
      self.model = model
      self.activations_list = ["relu","sigmoid"]

   def _get_layers(self):
      # dic = self.model.state_dict().items()
      # mod_list = []
      # mods = self.model.modules()
      # for mod in mods:
      #    mod_list.append(mod)
      architecture_dict = {}
      # # architecture = []
      layer = 0
      for _,module in enumerate(self.model):
         # if idx == 0:
         #       pass
         # else:
               # # Check if the module has children
               # # children is a generator
               # children = list(module.children())
               # num_children = len(children)
               # # print(f"Num children: {num_children}")
               # if num_children > 0:
               #    for child in children:
               #       architecture_dict[layer] =  str(child)
               #       # architecture.append(str(child))
               #       layer += 1
         architecture_dict[layer] =  str(module)
         layer += 1
      return architecture_dict

   def get_layer_signature(self,lines):
      lines = "torch.nn." + lines
      print(lines)
      layer = eval(lines)
      layer_name=layer.__class__
      layer_signature=inspect.signature(layer_name).parameters
      layer_signature_keys = layer_signature.keys()
      dict_keys = []
      for i in layer_signature_keys:
         dict_keys.append(i)
      return dict_keys

   def _signature(self,header):
      # (?<=\() : positive lookbehind to match the open bracket (
      # [\w\W] : any of \w(word,digit,whitespace) or \W(NOT word,digit,whitespace)
      #   s = re.findall('(?<=\()[\w\W]+(?=\)$)', header)
      s = re.findall('(?<=\().+(?=\))', header)
      # print(f"SIGNATURE:{s}")
      # print(s) = ['3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)']
      # ,\s(?=\w\w) : match , followed by \s(whitespace) and then (?=\w\W) [any of \w(word,digit,whitespace) \W(NOT word,digit,whitespace)]

      return re.split(',\s(?=\w\w)', s[0]) if s else ''

   def _extract_params(self,signature):
      params = []
      for each in signature:
         # Note this MUST be .match. It will not work if it is .search
         integers = re.match("\d+",each)
         if integers:
            params.append(integers.group())
            # print(f"Digit:{integers}")
         else:
            flt = re.search("(\d+\.\d+)",each)
            if flt:
               params.append(flt.group())
               # print(f"Float:{flt.group()}")
            else:
               scientific = re.search("(\d+[e])(\+|\-)\d+",each)
               if scientific:
                  params.append(scientific.group())
                  # print(f"Scientific:{scientific.group()}")
               else:
                  # string includes booleans
                  string = re.search("(?<=\=).+",each)
                  if string:
                     params.append(string.group())
                     # print(f"string(incl bool): {string.group()}")
                  else:
                     # get tuples
                     # Look for the = sign
                     # If the = sign exists, check for tuple
                     tuple = re.search("(?<=\=)\(.+\)",each)
                     if tuple:
                        params.append(tuple.group())
                        # print(tuple.group())
                        # print(f"Tuple: {tuple}")
      return params

   def parse_model(self):
      parsed_layers = []
      
      parameters = []
      architecture = self._get_layers()
      num_layers = len(architecture)
      # print(f"NUM LAYERS:{num_layers}")
      for idx in architecture:
         # print(f"INDEX:{idx}")
         param = {}
         module = {}
         layer_sig = self._signature(architecture[idx])
         param_value = self._extract_params(signature=layer_sig)
         param_name = self.get_layer_signature(architecture[idx])
         # print(f"param name: {param_name}")
         # print(params)
         module_name = re.match("^\w+",architecture[idx]).group()
         params = zip(param_name,param_value)
         # parsed_layers[module_name] = param_value
         for name,value in params:
            param[name] = eval(value)

         for each in self.activations_list:
            print(f"EACH:{each}")
            print(f"LOWER:{module_name.lower()}")
            if each in module_name.lower():
               module["activation"] = each
            else:
               module["activation"] = None
         if idx == 0:
            module["is_input"] = True
         
         # if param_name.lower() in self.activations_list:
         #    module["activation"] = param_name.lower()
         # else:
         #    module["activation"] = None
         
         module["l_type"] = module_name.lower()
         module["structure"] = param
         parsed_layers.append(module)
      
      return json.dumps(parsed_layers)

In [3]:
def get_children(model):
    # get children form model!
    children = list(model.children())
    flatt_children = []
    if children == []:
        # if model has no children; model is last child! :O
        return model
    else:
        # look for children from children... to the last child!
        for child in children:
            try:
                flatt_children.extend(get_children(model=child))
            except TypeError:
                flatt_children.append(get_children(model=child))
    return flatt_children

In [4]:
modelcfg = get_cfg_defaults()
model = HRNetRefine(modelcfg)
model_arch = get_children(model)

In [5]:
# def get_children(model: torch.nn.Module):
#     # get children form model!
#     children = list(model.children())
#     flatt_children = []
#     if children == []:
#         # if model has no children; model is last child! :O
#         return model
#     else:
#        # look for children from children... to the last child!
#        for child in children:
#             try:
#                 flatt_children.extend(get_children(child))
#             except TypeError:
#                 flatt_children.append(get_children(child))
#     return flatt_children

In [6]:
# model1 = get_children(model)
# model1

In [7]:
# layer = 0
# architecture_dict = {}
# for _,module in enumerate(model1):
#     print(module)
#     architecture_dict[layer] =  str(module)
#     layer += 1

# print(architecture_dict)

In [8]:
parser = ModelParser(model_arch)

layers_json = parser.parse_model()

TypeError: parse_model() missing 1 required positional argument: 'model'

In [ ]:
print(layers_json)

[{"activation": null, "is_input": true, "l_type": "conv2d", "structure": {"in_channels": 3, "out_channels": 64, "kernel_size": [3, 3], "stride": [1, 1], "padding": [1, 1], "dilation": false}}, {"activation": null, "l_type": "batchnorm2d", "structure": {"num_features": 64, "eps": 1e-05, "momentum": 0.1, "affine": true, "track_running_stats": true}}, {"activation": null, "l_type": "conv2d", "structure": {"in_channels": 64, "out_channels": 64, "kernel_size": [3, 3], "stride": [1, 1], "padding": [1, 1], "dilation": false}}, {"activation": null, "l_type": "batchnorm2d", "structure": {"num_features": 64, "eps": 1e-05, "momentum": 0.1, "affine": true, "track_running_stats": true}}, {"activation": null, "l_type": "relu", "structure": {"inplace": true}}, {"activation": null, "l_type": "conv2d", "structure": {"in_channels": 64, "out_channels": 64, "kernel_size": [1, 1], "stride": [1, 1], "padding": false}}, {"activation": null, "l_type": "batchnorm2d", "structure": {"num_features": 64, "eps": 1e